In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy.ndimage import gaussian_filter

plt.rcParams.update({'font.size': 14})

In [ ]:
import os

In [ ]:
DATA_FOLDER = "data"
WAV_FOLDER = os.path.join(DATA_FOLDER, "music", "wav")

WAV_FILES = list(map(lambda x: os.path.join(WAV_FOLDER, x), os.listdir(WAV_FOLDER)))[0:1]

In [ ]:
MSIZE = 256

SIGMA = [7.0, 7.0]

In [ ]:
for wav in tqdm(WAV_FILES):
    matrix = np.zeros((MSIZE, MSIZE), dtype="uint32")
    with open(wav, "rb") as f:
        raw_data = f.read()
    
    n = len(raw_data)
    
    for i in range(n-1):
        idx1 = raw_data[i]
        idx2 = raw_data[i+1]
        
        matrix[idx1, idx2] += 1

In [ ]:
res_matrix = gaussian_filter(
    matrix,
    SIGMA,
    mode="constant",
    cval=matrix.mean()
)

matrix.tofile("test.bin")
hex(matrix[0,0])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,6), dpi=120)
ax.grid()
# ax.set_xlim(1, 70)
# ax.set(xlabel='common xlabel', ylabel='common ylabel')

sns.heatmap(data=res_matrix)